In [ ]:
import re
import json

from simple_websocket_server import WebSocketServer, WebSocket

class Controller: 
    def process(self, telemetry):
        print(telemetry)
        return {}
    

class SimulatorClient(WebSocket):
    incoming_pattern = re.compile('42\["telemetry",\s*(.*)\s*\]')
    response_template = '42["manual",%s]'
    
    def connected(self):
        self.controller = Controller()
        
    def handle(self):
        command = self._receive_telemetry(self.data)
        self._send_command(command)
        
    def _receive_telemetry(self, package):
        match_data = self.incoming_pattern.match(package)
        if not match_data: return None
        
        telemetry = json.loads(match_data.group(1))
        return self.controller.process(telemetry)
            
    def _send_command(self, command):
        response = self.response_template % json.dumps(command)
        self.send_message(response)
        

class UdacitySimulator: 
    def run(self):
        server = WebSocketServer('', 4567, SimulatorClient)
        try:
            server.serve_forever()
        except KeyboardInterrupt: 
            server.close()


In [ ]:
sim = UdacitySimulator()
sim.run()